In [40]:
from gerrychain import Graph, Partition, Election, GeographicPartition
from gerrychain.updaters import Tally, cut_edges
import geopandas
import pandas
import maup
from matplotlib import pyplot as plt
import json
import shapely

In [2]:
election = geopandas.read_file("./tennessee/tn_election_block.zip").to_crs(32030)

In [3]:
census = geopandas.read_file("./tennessee/tn_census_vtd.zip").to_crs(32030)

In [5]:
cleaned_election = geopandas.GeoDataFrame(election[["GEOID20", 
                                                   "G20PRERTRU", 
                                                   "G20PREDBID", 
                                                   "geometry"]])

In [10]:
cleaned_census = geopandas.GeoDataFrame(census[["NAME20", 
                                                "P0020001",
                                                "P0020002",
                                                "P0020005",
                                                "P0020006",
                                                "geometry"]])

In [18]:
# BLOCK LEVEL
renamed_election = cleaned_election.rename(columns={"G20PRERTRU": "TRUMP20",
                                                    "G20PREDBID": "BIDEN20"
}, errors="raise")

In [19]:
# VTD LEVEL
renamed_census = cleaned_census.rename(columns={"P0020001": "TOTPOP20",
                                                "P0020002": "HISPLAT20",
                                                "P0020005": "WHITE20",
                                                "P0020006": "BLACKAA20"
}, errors="raise")

In [22]:
# variables = ["TOTPOP20", "HISPLAT20", "WHITE20", "BLACKAA20"]
variables = ["TRUMP20", "BIDEN20"]
blocks_to_vtd_assignment = maup.assign(renamed_election, renamed_census)
renamed_census[variables] = renamed_election[variables].groupby(blocks_to_vtd_assignment).sum()
renamed_census[variables].head()

,TRUMP20,BIDEN20
0,958.04,1224.99
1,504.00,704.96
2,205.00,476.01
3,470.00,297.02
4,464.03,203.01


In [24]:
rows_to_drop = []
for i, row in renamed_census.iterrows():
    if row.geometry is None:
        rows_to_drop.append(i)
        continue
    if row["NAME20"] == "Voting Districts Not Defined":
        rows_to_drop.append(i)
final_gdf = renamed_census.drop(rows_to_drop)

In [26]:
districts = geopandas.read_file("./tennessee/tn_houserep_districts.zip").to_crs(32030)

In [27]:
vtd_to_district_assignment = maup.assign(final_gdf, districts)
final_gdf["HR_DIST"] = vtd_to_district_assignment
vtd_to_district_assignment.head()

0    26
1    26
2    26
3    26
4    26
dtype: int64

In [28]:
final_gdf.to_file("tennessee.geojson", driver="GeoJSON")

In [38]:
districts = max(final_gdf["HR_DIST"]) + 1

In [41]:
geometries = []
district_data = []
for district in range(0, districts):
    filtered_gdf = final_gdf[final_gdf["HR_DIST"] == district]
    district_geometry = shapely.unary_union(filtered_gdf["geometry"])
    geometries.append(district_geometry)
    
    # Sum up election and demographic data
    exclude = ["NAME20", "HR_DIST", "geometry"]
    selected_gdf = filtered_gdf.drop(columns=exclude)
    sum_data = selected_gdf.sum()
    sum_data["district_id"] = district
    district_data.append(sum_data)

df_list = []
for series, geom in zip(district_data, geometries):
    df = pandas.DataFrame(series).T
    df["geometry"] = geom
    df_list.append(df)

concat_df = pandas.concat(df_list)
og_gdf = geopandas.GeoDataFrame(concat_df, geometry="geometry")

In [44]:
with open("district_tennessee.geojson", "w") as f:
    f.write(og_gdf.to_json())